In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.config import settings
from pathlib import Path

In [3]:
# See: https://github.com/michalc/sqlite-s3-query

from sqlite_s3_query import sqlite_s3_query

In [4]:
def get_scaleway_credentials(_):

    # REGION_NAME = settings.REGION_NAME
    # ENDPOINT_URL = settings.ENDPOINT_URL

    SECRETS_FILE = ".secrets.toml"

    if (
        Path(SECRETS_FILE).exists() is True
        or Path(f'../{SECRETS_FILE}').exists() is True  # if being run from notebooks subdirectory
    ):    # if there is a local secrets file
        SECRET_ACCESS_KEY = settings.SCALEWAY_SECRET_ACCESS_KEY
    else:                                      # use environment variables (deployed on Streamlit Sharing)
        SECRET_ACCESS_KEY = os.environ["SCALEWAY_SECRET_ACCESS_KEY"]


    return (
        settings.REGION_NAME,
        settings.ACCESS_KEY_ID,
        SECRET_ACCESS_KEY,
        None
    )

In [5]:
# get_scaleway_credentials(_)

In [6]:
bucket_name = "databooth-beach-swim"
region_name = settings.REGION_NAME
database_name = "daily_beach_data_db.sqlite"

SQLITE_DB_URL = f"https://{bucket_name}.s3.{region_name}.scw.cloud/{database_name}"

In [7]:
SQLITE_DB_URL

'https://databooth-beach-swim.s3.fr-par.scw.cloud/daily_beach_data_db.sqlite'

In [8]:
eg_query = 'SELECT "Beach name" FROM beaches WHERE Region = "SydneyOceanBeaches"'

eg_query = eg_query.replace("SydneyOceanBeaches", "?, params=('SydneyOceanBeaches', ))")


In [9]:
eg_query

'SELECT "Beach name" FROM beaches WHERE Region = "?, params=(\'SydneyOceanBeaches\', ))"'

In [10]:
eg_query = 'SELECT "Beach name" FROM beaches WHERE Region = ?, params=("SydneyOceanBeaches", )'

In [12]:
eg_query

'SELECT "Beach name" FROM beaches WHERE Region = ?, params=("SydneyOceanBeaches", )'

In [11]:

with sqlite_s3_query(url=SQLITE_DB_URL, get_credentials=get_scaleway_credentials) as query:
    with query(eg_query) as (columns, rows):
        for row in rows:
            print(row)


 # WHERE index = ?', params=('2', )) as (columns, rows):   <-- not working yet - see examples


Exception: near ",": syntax error